In [1]:
# First, install required packages
!pip install google-auth-oauthlib google-auth-httplib2 google-api-python-client markdown

from google.colab import auth
from google.colab import drive
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import markdown
import re
import json

# Mount Google Drive
drive.mount('/content/drive')

class MarkdownToGoogleDocs:
    def __init__(self):
        self.service = None
        self.current_index = 1

    def authenticate(self):
        try:
            auth.authenticate_user()
            self.service = build('docs', 'v1', cache_discovery=False)
            return True
        except Exception as e:
            print(f"Authentication failed: {e}")
            return False

    def create_document(self, title):
        try:
            document = self.service.documents().create(body={'title': title}).execute()
            print(f"Created document with title: {title}")
            return document.get('documentId')
        except HttpError as error:
            print(f'An error occurred: {error}')
            return None

    def clean_line(self, line):
        """Clean a single line of markdown text and remove leading markdown markers."""
        # Remove leading spaces to prevent indentation from text itself
        line = line.lstrip()

        # Remove heading markers
        if line.startswith('# '):
            return line[2:]
        elif line.startswith('## '):
            return line[3:]
        elif line.startswith('### '):
            return line[4:]

        # Remove list markers
        if line.startswith('- [ ]'):
            return line[5:]
        elif line.startswith('* '):
            return line[2:]
        elif line.startswith('- '):
            return line[2:]

        return line

    def process_document(self, markdown_text):
        """Process markdown text and generate formatting requests."""
        lines = markdown_text.split('\n')
        processed_lines = []
        formatting_requests = []
        current_index = 1

        # Store list item information
        bullet_lines = []
        # Store the checkbox lines
        checkbox_lines = []

        for i, original_line in enumerate(lines):
            # Calculate indentation level
            indent_level = (len(original_line) - len(original_line.lstrip())) // 2
            stripped_orig = original_line.strip()

            # Determine list type
            is_list_item = False
            is_checkbox = False
            if stripped_orig.startswith('- [ ]'):
                is_checkbox = True
                is_list_item = True
            elif stripped_orig.startswith('* ') or stripped_orig.startswith('- '):
                is_list_item = True

            cleaned_line = self.clean_line(original_line)

            line_start = current_index
            line_end = line_start + len(cleaned_line)

            # Add special marking for checkbox items
            if is_checkbox:
                formatting_requests.append({
                    'createParagraphBullets': {
                        'range': {
                            'startIndex': line_start,
                            'endIndex': line_end
                        },
                        'bulletPreset': 'BULLET_CHECKBOX'
                    }
                })

            # Record processed line information
            processed_lines.append({
                'text': cleaned_line,
                'start': line_start,
                'end': line_end,
                'original': original_line,
                'indent_level': indent_level,
                'is_list_item': is_list_item,
                'is_checkbox': is_checkbox
            })

            # Process heading styles
            if stripped_orig.startswith('# '):
                formatting_requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': line_start, 'endIndex': line_end},
                        'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                        'fields': 'namedStyleType'
                    }
                })
            elif stripped_orig.startswith('## '):
                formatting_requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': line_start, 'endIndex': line_end},
                        'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                        'fields': 'namedStyleType'
                    }
                })
            elif stripped_orig.startswith('### '):
                formatting_requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': line_start, 'endIndex': line_end},
                        'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                        'fields': 'namedStyleType'
                    }
                })

            # Record non-checkbox list items
            if is_list_item and not is_checkbox:
                bullet_lines.append((line_start, line_end, indent_level))

            # Process @mentions
            mentions = re.finditer(r'@\w+', cleaned_line)
            for mention in mentions:
                mention_start = line_start + mention.start()
                mention_end = line_start + mention.end()
                formatting_requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': mention_start,
                            'endIndex': mention_end
                        },
                        'textStyle': {
                            'bold': True,
                            'foregroundColor': {
                                'color': {'rgbColor': {'blue': 0.8, 'red': 0.2, 'green': 0.2}}
                            }
                        },
                        'fields': 'bold,foregroundColor'
                    }
                })

            # Process footer
            if any(x in cleaned_line for x in ["Meeting recorded by:", "Duration:"]):
                formatting_requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': line_start, 'endIndex': line_end},
                        'textStyle': {
                            'italic': True,
                            'foregroundColor': {'color': {'rgbColor': {'blue': 0.5, 'red': 0.5, 'green': 0.5}}}
                        },
                        'fields': 'italic,foregroundColor'
                    }
                })

            current_index = line_end + 1

        # Process formatting requests for list items
        bullet_requests = []
        for (start, end, indent_level) in bullet_lines:
            # Create list item
            bullet_requests.append({
                'createParagraphBullets': {
                    'range': {
                        'startIndex': start,
                        'endIndex': end
                    },
                    'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
                }
            })

            # Handle indentation for multilevel lists
            if indent_level > 0:
                bullet_requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': start, 'endIndex': end},
                        'paragraphStyle': {
                            'indentStart': {
                                'magnitude': 36 * indent_level,
                                'unit': 'PT'
                            },
                            'indentFirstLine': {
                                'magnitude': 36 * indent_level,
                                'unit': 'PT'
                            }
                        },
                        'fields': 'indentStart,indentFirstLine'
                    }
                })

        # Merge all formatting requests, ensuring list processing comes first
        formatting_requests = bullet_requests + formatting_requests

        # Return processed text and formatting requests
        return '\n'.join(line['text'] for line in processed_lines), formatting_requests


    def update_document(self, document_id, markdown_text):
        try:
            # Process the document
            cleaned_text, formatting_requests = self.process_document(markdown_text)

            # First, insert the cleaned text
            self.service.documents().batchUpdate(
                documentId=document_id,
                body={'requests': [{'insertText': {'location': {'index': 1}, 'text': cleaned_text}}]}
            ).execute()

            # Apply formatting in batches
            batch_size = 20
            for i in range(0, len(formatting_requests), batch_size):
                batch_requests = formatting_requests[i:i + batch_size]
                self.service.documents().batchUpdate(
                    documentId=document_id,
                    body={'requests': batch_requests}
                ).execute()

            print("Document updated successfully!")
            return True

        except HttpError as error:
            print(f'An error occurred: {error}')
            return False

# Your markdown text
markdown_text = """# Product Team Sync - May 15, 2023
## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)
## Agenda
### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration
### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues
### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates
## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal
## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders
## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""

def main():
    try:
        converter = MarkdownToGoogleDocs()

        if not converter.authenticate():
            raise Exception("Authentication failed")

        doc_id = converter.create_document("Product Team Sync Notes")
        if not doc_id:
            raise Exception("Failed to create document")

        success = converter.update_document(doc_id, markdown_text)

        if success:
            print(f"Document created successfully! ID: {doc_id}")
            print(f"You can view your document at: https://docs.google.com/document/d/{doc_id}/edit")
        else:
            print("Failed to update document")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Mounted at /content/drive
Created document with title: Product Team Sync Notes
Document updated successfully!
Document created successfully! ID: 1un0I30h8wiifJ8QZt8uscjTpbuGXj1eiJz8qzcNSpvE
You can view your document at: https://docs.google.com/document/d/1un0I30h8wiifJ8QZt8uscjTpbuGXj1eiJz8qzcNSpvE/edit
